<a href="https://colab.research.google.com/github/taiynlee/Kubernetes-CN/blob/master/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd

In [7]:
train_df=pd.read_csv("train.csv",encoding="utf-8")
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [8]:
test_df=pd.read_csv("test.csv",encoding="utf-8")
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


資料分二種 數值和類別
預處理
一、補缺失值 類別用最常出現的值(眾數) 數值依不同情況 取平均數或中位數 中位數可以去除最高與最低的極端值 絕大部份都要用中位數 (比如就人來說用中位數 就錢來說用平均數)
二、類別 onehotencoding

In [9]:
train_df.isna()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
886,False,False,False,False,False,False,False,False,False,False,True,False
887,False,False,False,False,False,False,False,False,False,False,False,False
888,False,False,False,False,False,True,False,False,False,False,True,False
889,False,False,False,False,False,False,False,False,False,False,False,False


In [10]:
s=train_df.isna().sum()

In [11]:
s>0

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool

In [12]:
s[s>0]
# filter:帶入跟資料大小一樣的true/false,ture留,false去除

Age         177
Cabin       687
Embarked      2
dtype: int64

In [13]:
s[s>0].sort_values(ascending=False)

Cabin       687
Age         177
Embarked      2
dtype: int64

In [14]:
s=test_df.isna().sum()
s[s>0].sort_values(ascending=False)

Cabin    327
Age       86
Fare       1
dtype: int64

In [15]:
train_df.median()

PassengerId    446.0000
Survived         0.0000
Pclass           3.0000
Age             28.0000
SibSp            0.0000
Parch            0.0000
Fare            14.4542
dtype: float64

In [16]:
med=train_df.median()
med=med.drop(["Pclass"])
train_df=train_df.fillna(med)
test_df=test_df.fillna(med)

In [17]:
s=train_df.isna().sum()
s[s>0]

Cabin       687
Embarked      2
dtype: int64

In [18]:
s=test_df.isna().sum()
s[s>0]

Cabin    327
dtype: int64

In [19]:
# 類別形態:最常出現
train_df["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [20]:
counts=train_df["Embarked"].value_counts()
most=counts.idxmax()
train_df["Embarked"]=train_df["Embarked"].fillna(most)

In [21]:
s=train_df.isna().sum()
s[s>0]

Cabin    687
dtype: int64

In [22]:
train_df["Name"]

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
886                                Montvila, Rev. Juozas
887                         Graham, Miss. Margaret Edith
888             Johnston, Miss. Catherine Helen "Carrie"
889                                Behr, Mr. Karl Howell
890                                  Dooley, Mr. Patrick
Name: Name, Length: 891, dtype: object

In [23]:
n="Braund, Mr. Owen Harris"
n=n.split(",")[1].split(".")[0]
n.strip()

'Mr'

In [24]:
import re
n="Braund, Mr. Owen Harris"
pat=r"\w+,\s*(\w+)\.[\s\w]+"
# \w:A-Z a-z  \s:blank  +:>=1  *>=0 \w+: Braund:一個以上a-z  \s*:零個以上的空白  (\+):Mr, ()是指這是我的target  \. .是有特殊用途的所以我\. 就是要抓.
# [\s\w]+: owen harris \s or \w 出現一次以上
# group:找出()
re.search(pat,n).group(1)

'Mr'

In [25]:
def namecut(n):
  reserved=["Mr","Miss","Mrs","Master"]
  s=n.split(",")[-1].split(".")[0].strip()
  if s in reserved:
    return s
  


In [26]:
train_df["Name"]=train_df["Name"].apply(namecut)
train_df["Name"]

0        Mr
1       Mrs
2      Miss
3       Mrs
4        Mr
       ... 
886    None
887    Miss
888    Miss
889      Mr
890      Mr
Name: Name, Length: 891, dtype: object

In [27]:
test_df["Name"]=test_df["Name"].apply(namecut)
x_train=train_df.drop(["PassengerId","Survived","Ticket","Cabin"],axis=1)
x_test=test_df.drop(["PassengerId","Ticket","Cabin"],axis=1)

In [28]:
# onehot
pd.get_dummies(x_train)

,Pclass,Age,SibSp,Parch,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,0,0,1,0,0,1,0,0,1
1,1,38.0,1,0,71.2833,0,0,0,1,1,0,1,0,0
2,3,26.0,0,0,7.9250,0,1,0,0,1,0,0,0,1
3,1,35.0,1,0,53.1000,0,0,0,1,1,0,0,0,1
4,3,35.0,0,0,8.0500,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,0,0,0,0,0,1,0,0,1
887,1,19.0,0,0,30.0000,0,1,0,0,1,0,0,0,1
888,3,28.0,1,2,23.4500,0,1,0,0,1,0,0,0,1
889,1,26.0,0,0,30.0000,0,0,1,0,0,1,1,0,0
